In [31]:
# 랭체인에서 RAG(검색 + 생성) 구현, 분기체인(규칙위반 멀티체인 선택)
!pip install langchain  langchain-google-genai sentence-transformers python-dotenv langchain-chroma langchain-community


ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7d3a4ce42690>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 126, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 99, in _retrieve_info
    info = _metadata.get_service_account_info(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 338, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 263, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http:/

ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7d3a4ce42690>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 126, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 99, in _retrieve_info
    info = _metadata.get_service_account_info(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 338, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 263, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http:/

 * LangChain이라는 강력한 프레임워크를 사용하여 인공지능(AI) 모델을 활용하는 두 가지 핵심 기술 (검색 증강 생성(RAG)과 모델의 창의성 제어)

1) LangChain을 활용하여 대규모 언어 모델(LLM)인 Gemini와 벡터 데이터베이스를 연결하는 방법.

  1-1) 검색 증강 생성 (RAG) 구현: 자체 문서를 LLM에 연결하여, 모델이 학습하지 않은 최신 정보나 내부 기밀 정보를 바탕으로 정확한 답변을 생성하도록 하는 기술.
RAG 구조는 문서 저장 → 검색(Retrieval) → 답변 생성(Generation)의 흐름
  
  1-2)멀티 실행 체인 및 Temperature 활용: LLM을 호출할 때 temperature라는 매개변수를 조정하여, 논리적이고 안정적인 답변과 창의적이고 다양한 답변을 동시에 얻어내는 방법을 실습

In [32]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document                     # 랭체인에서 문서를 다루는 기본 형식
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma               # 벡터 DB, 문서를 숫자로 변환하여 저장
from langchain_community.embeddings import HuggingFaceEmbeddings  # 임베딩 모델 (문서를 벡터로 변환)
import os
from dotenv import load_dotenv
from google.colab import userdata
load_dotenv()
os.environ["GOOGLE_GENAI_PROJECT"] = userdata.get('GOOGLE_GENAI_PROJECT')

# LLM 모델
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7, google_api_key=userdata.get('GOOGLE_API_KEY'))
# 임베딩 모델
# (과금)
# embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

#모델이 문서를 '벡터'라는 숫자 형태로 변환해 Chroma DB에 저장
# 과금없는 모델
embedding_model = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
)

# 문서 데이터 준비
# 랭체인 RAG(Retrieval Augmented Generation_검색증강생성)구조는 문서들을 저장 -> 검색 -> 답변 생성하는 흐름을 갖음
# 파일(.txt, .pdf...)을 읽으면 구조가 복잡해 일단 메모리에 문서데이터를 만든다. (document type)
docs = [
    Document(page_content="박 검사장은 지난 19일 법무부 검찰 고위 간부 인사를 통해 서울중앙지검장으로 새로 임명됐다. 정진우 전 서울중앙지검장이 대장동 항소 포기 사태 이후 사의를 표한 지 11일 만이었다.박 검사장은 당시 대검찰청 반부패부장으로서 항소 포기 결정 과정에 깊이 관여했다. 박 검사장은 대장동 1심 선고 이후 법무부 측으로부터 '신중 검토 필요' 의견을 전달받은 뒤, 항소한다는 입장이던 서울중앙지검 수사팀에 재검토를 지휘한 것으로 전해졌다. 당시 수사팀은 박 검사장의 지휘를 사실상 '항소 불허'로 받아들였다는 입장이다. 실제 정진우 전 검사장은 사의를 표명하면서 \"대검의 지휘를 수용하지만, 중앙지검의 의견이 다르다는 점을 명확히 한다\"고 언급한 바 있다.검찰 안팎에서는 항소 포기 사태 지휘라인에 있던 박 검사장이 대장동 수사·공판팀을 이끌게 된 만큼, 조직 안정 및 장악이 쉽지 않을 것이라는 우려도 제기된다."),
    # Document(page_content="") # 주제가 다르면 document를 따로따로 만들어야 정확도가 올라감
]
# 텍스트를 조각으로 쪼개기 (옵션)

text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
split_docs = text_splitter.split_documents(docs)

# 랭체인이 지원하는 Chromadb에 저장
db = Chroma.from_documents(split_docs, embedding_model)

# Retriver
# 저장된 DB에서 질문과 가장 유사한 문서 조각을 '검색(Retrieve)'할 수 있는 도구를 만듬
retriver = db.as_retriever()

# PromptTemplate 작성
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template = """
      너는 친절하고 똑똑한 AI어시스턴트야.
      아래 문서 내용을 참조해 나의 질문에 정확하게 답을 해줘.
      문서 내용일 불충분한 경우 '문서에 해당 정보가 없어요'라고 답변해.
      문맥 :
      {context}
      질문 :
      {question}
      답변은 5행 정도만 해줘.
    """
)

# 체인 생성(LCEL방식)
def format_docsFunc(docs):
  # 검색된 document 리스트를 하나의 문자열로 문자열을 합쳐 반환하는 헬퍼함수
  return "\n\n".join(doc.page_content for doc in docs)
# 전체 RAG 흐름을 하나의 파이프라인으로 연결합니다. (LangChain Expression Language)
# 질문(question)이 들어오면, Retriever를 통해 문맥(context)을 생성하고, 이를 Prompt에 넣어 LLM을 호출하는 구조
rag_chain = (
    {
        "context":retriver | format_docsFunc, #질문 -> retriver로 검색 -> 문서들을 문자열로 포맷
        "question":RunnablePassthrough()      #원래 질문은 그대로 전달
    }
    | prompt_template
    | llm
    | StrOutputParser()
)

# 질문
query = "검찰청에 무슨일이 있었던거야?"
result = rag_chain.invoke(query)
print('질문 : ', query)
print('답변 : ', result)

ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7d3a4ce42690>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 126, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 99, in _retrieve_info
    info = _metadata.get_service_account_info(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 338, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 263, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http:/

질문 :  검찰청에 무슨일이 있었던거야?
답변 :  검찰청에서는 다음과 같은 일이 있었습니다.

정진우 전 서울중앙지검장이 대장동 항소 포기 사태 이후 사의를 표명했습니다. 이에 박 검사장이 서울중앙지검장으로 새로 임명되었습니다.

박 검사장은 과거 대검찰청 반부패부장으로서 대장동 1심 선고 후 항소를 준비하던 서울중앙지검 수사팀에 '재검토'를 지휘했습니다. 수사팀은 이를 사실상 '항소 불허'로 받아들였고, 정진우 전 지검장도 대검의 지휘를 수용하지만 중앙지검의 의견이 달랐음을 명확히 했습니다.

이러한 배경 때문에, 항소 포기 사태 지휘라인에 있던 박 검사장이 대장동 수사·공판팀을 이끌게 되면서 조직 안정 및 장악이 쉽지 않을 것이라는 우려가 제기되고 있습니다.


In [33]:
# LLM의 답변 성향을 제어하는 핵심적인 방법
# 멀티 실행 체인  : RunnalbeParallel을 사용해 여러 체인을 병렬로 처리
# 여건상 RunnableParallel 사용 불가. 그래서 여기서는 순차적으로 두 번 LLM 호출
llm_calm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1, google_api_key=userdata.get('GOOGLE_API_KEY'))
llm_creative = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.9, google_api_key=userdata.get('GOOGLE_API_KEY'))
q = "AI가 뭔가요?"
out1 = llm_calm.invoke(q)
out2 = llm_creative.invoke(q)
print("1) 현실적인 답변 : ", out1.content)
print("2) 창의적인 답변 : ", out2.content)


ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7d3a4ce42690>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 126, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 99, in _retrieve_info
    info = _metadata.get_service_account_info(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 338, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 263, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http:/

1) 현실적인 답변 :  인공지능(AI, Artificial Intelligence)은 한마디로 **컴퓨터가 사람처럼 생각하고, 배우고, 판단하고, 행동하게 만드는 기술 또는 분야**를 말합니다.

조금 더 자세히 설명하자면 다음과 같습니다:

1.  **인간 지능의 모방:** AI의 목표는 인간이 가진 지능적인 능력, 예를 들어 학습, 추론, 문제 해결, 지각, 언어 이해, 창작 등을 컴퓨터 시스템이 수행할 수 있도록 하는 것입니다.

2.  **데이터 기반 학습:** 대부분의 현대 AI는 방대한 양의 데이터를 분석하고 거기서 패턴을 찾아 스스로 학습합니다. 이를 통해 새로운 상황이나 문제에 대해 적절한 결정을 내리거나 예측을 할 수 있게 됩니다.

3.  **다양한 형태:** AI는 특정 작업을 아주 잘 수행하는 '약 인공지능'(예: 바둑 AI, 음성 비서)부터, 아직은 이론적인 단계이지만 인간처럼 모든 분야에서 지능적인 능력을 발휘하는 '강 인공지능'까지 다양한 형태로 발전하고 있습니다.

**쉽게 비유하자면:**
AI는 컴퓨터에게 '뇌'를 만들어주는 것과 비슷합니다. 이 뇌는 데이터를 통해 배우고, 경험을 쌓아 더 똑똑해지며, 주어진 문제를 해결하거나 새로운 것을 만들어낼 수 있게 됩니다.

**일상생활 속 AI의 예시:**

*   **음성 비서:** 시리(Siri), 빅스비(Bixby), 구글 어시스턴트 등 (사람의 말을 이해하고 명령을 수행)
*   **추천 시스템:** 넷플릭스, 유튜브, 온라인 쇼핑몰 (사용자의 취향을 분석해 콘텐츠나 상품 추천)
*   **자율주행차:** 주변 환경을 인식하고 스스로 운전 (인지, 판단, 제어)
*   **번역 앱:** 다른 언어를 실시간으로 번역 (언어 이해 및 생성)
*   **스팸 메일 필터:** 불필요한 메일을 자동으로 분류 (패턴 인식)
*   **챗봇:** 저처럼 질문에 답하고 대화하는 시스템 (자연어 처리)
*   **얼굴 인식/지문 인식:** 스마트폰 잠금 해제, 보안 시스템 (이미지 인식)

결

 1. RAG의 '데이터 처리' 핵심 요소 깊이 이해하기

     RAG 체인의 성공은 LLM 성능 20%, 나머지 80%는 데이터를 어떻게 처리했는지에 달려 있습니다. 특히 임베딩, 분할, 벡터 저장소의 역할에 집중.

        1-1) 임베딩 모델 (Embedding Model)의 역할
              HuggingFaceEmbeddings이 하는 역할은 단순한 변환이 아님.
              
              배워야 할 세부 내용:"벡터란 무엇인가?" 문장의 의미를 수백 차원의 숫자 배열로 표현하는 방법.
              임베딩의 품질: 왜 특정 임베딩 모델(예: all-MiniLM-L6-v2)을 선택했는지, 다른 모델을 사용하면 결과가 어떻게 달라지는지 이해해야 함. 임베딩 모델은 의미론적 유사성을 얼마나 잘 포착하느냐가 핵심.
              데이터 업무 연결: 기업의 데이터(업무 보고서, 규정집 등)를 분석할 때, 이 임베딩 모델이 데이터 간의 관계를 정의하는 가장 중요한 도구.

        1-2) 텍스트 분할기 (Text Splitter)의 중요성 CharacterTextSplitter(chunk_size=200, chunk_overlap=20)를 사용했는데 이 숫자가 검색 품질을 결정.
        
              배워야 할 세부 내용:chunk_size의 최적화: 왜 200이 적절한지? 500이나 50으로 바꾸면 어떤 문제가 생길까? 너무 크면 LLM이 불필요한 정보까지 받아 응답이 부정확해지고, 너무 작으면 문맥(Context)이 끊겨 의미 파악이 안 됨.
              chunk_overlap의 역할: 중복되는 20글자(Overlap)는 왜 필요한가요? 이는 분할된 문서 조각(Chunk) 사이의 문맥 연속성을 확보하여, 검색 시 핵심 정보가 조각의 경계에 걸쳐 있더라도 놓치지 않도록!.
              
              다른 분할 전략: 문자를 기준으로 자르는 CharacterTextSplitter 외에, 문단, 문장, 코드를 인식하여 자르는 다른 분할기(예: RecursiveCharacterTextSplitter)가 왜 필요한지 비교하며 학습해야 함.
        
        1-3) 벡터 데이터베이스 (Chroma)의 기능Chroma.from_documents를 사용하여 벡터 DB에 저장.

              
              배워야 할 세부 내용:기존 DB와의 차이점: SQL을 사용하는 관계형 DB(RDB)가 '정확한 값'(예: 이름='홍길동')을 찾는다면, 벡터 DB는 '가장 비슷한 의미'를 갖는 데이터를 찾음.
              데이터 업무를 하려면 두 DB의 특성과 사용 시점을 명확히 구분해야 함.
              검색(Retrieval) 원리: Chroma가 질문 벡터와 저장된 문서 벡터 간의 유사도(Similarity)를 어떻게 측정하는지(예: 코사인 유사도) 원리를 학습
        
  2. RAG 체인을 '데이터 파이프라인'으로 확장하기 RAG 구축 경험은 데이터 엔지니어링의 핵심인 ETL (Extract-Transform-Load) 파이프라인과 매우 유사.
  
          ETL 단계 - RAG 파이프라인 단계 - 학습 연관성
          Extract (추출) - 문서 수집 (Document 생성) - 전산 시스템에서 로그/텍스트 데이터 추출
          Transform (변환) - 텍스트 분할, 임베딩(CharacterTextSplitter, Embedding) - 파이썬 Pandas를 이용한 데이터 정제 및 가공
          Load (적재) - 벡터 DB 저장 (Chroma.from_documents) - SQL을 이용한 전통 DB 또는 벡터 DB에 데이터 적재
    
  
      심화 학습 주제: 데이터 가공 (Transform)RAG 시스템에 넣기 전에 파이썬 Pandas를 사용해서 문서 데이터를 정제하는 방법. (예: 불필요한 특수문자 제거, 중복 문서 제거, 오탈자 수정 등) 깨끗한 데이터가 정확한 RAG 결과를 만든다!
  
  3. LLM 제어 및 활용 심화: Prompt Engineering단순히 temperature를 조절하는 것을 넘어, LLM의 답변 품질과 성향을 명시적으로 제어하는 기술
      1) 프롬프트 엔지니어링 (Prompt Engineering) 심화파일에서 PromptTemplate을 사용했는데 템플릿에 어떤 내용을 넣느냐에 따라 AI의 업무 성과가 결정됨.


          배워야 할 세부 내용:
          
          역할 부여 (Persona): 프롬프트 설정을 특정 업무 역할을 부여하는 연습.
          사고 과정 유도 (Chain-of-Thought, CoT): LLM에게 최종 답변을 내기 전에 "먼저 단계별로 분석하고 그 과정을 나열한 뒤, 최종 답변을 해라"와 같이 사고의 절차를 요구하여 답변의 논리성과 정확성을 높이는 방법으로 학습.
  
      2) LangChain 심화 체인 구조파일에서는 가장 기본적인 RAG 체인을 구성.
          
          배워야 할 세부 내용 :
          
          라우터(Router) 체인: 사용자의 질문을 분석하여, 데이터 분석이 필요한 질문은 RAG 체인으로 보내고, 일반 상식 질문은 일반 LLM으로 바로 보내도록 분기하는 구조를 만들어보기. 이는 복잡한 실무 환경에서 AI의 효율을 극대화하는 방법.
          라우터(Router) 체인은 실무 환경에서 AI 시스템의 효율성과 비용을 획기적으로 개선할 수 있는 매우 중요한 기술임. 마치 AI 시스템의 지능적인 길 안내자

         라우터(Router) 체인이란? : 라우터 체인은 들어오는 사용자의 질문(Query)을 분석하여, 이 질문에 답변하기 위해 가장 적합한 하위 체인(Sub-Chain) 또는 도구(Tool)로 요청을 분기(Routing)시키는 역할을 하는 상위 체인(Master Chain)

            단계별 원리 : 라우팅 LLM 정의: 질문을 분석하고 어떤 경로로 보낼지 결정할 LLM을 정의.
            분기 프롬프트 (Router Prompt): 이 LLM에게 "이 질문은 RAG에 적합한가? 아니면 일반 상식에 적합한가?"를 판단하도록 명시적으로 지시하는 프롬프트를 작성.
            경로 선택 (Route Selection): 라우팅 LLM의 답변(예: "RAG")을 분석하여 실제 실행할 하위 체인을 선택.
            하위 체인 실행: 선택된 하위 체인(RAG 또는 LLM)을 실행


In [38]:
# RAG 체인 재구성을 위한 모듈 및 객체 정의
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. LLM 정의 (router_llm, normal_llm_chain에 사용)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7, google_api_key=userdata.get('GOOGLE_API_KEY'))

# 2. 임베딩 및 DB 정의
embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
docs = [Document(page_content="박 검사장은 지난 19일 법무부 검찰 고위 간부 인사를 통해 서울중앙지검장으로 새로 임명됐다. 그는 대장동 개발사업 의혹의 수사를 이끌었다. 다만, 조직 안정 및 장악이 쉽지 않을 것이라는 우려도 제기된다.")]
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
split_docs = text_splitter.split_documents(docs)
db = Chroma.from_documents(split_docs, embedding_model)
retriver = db.as_retriever() # 검색기 정의

# 3. format_docsFunc 정의 (검색된 문서를 문자열로 합치기)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 4. RAG 프롬프트 정의
rag_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template = "아래 문서 내용을 참조해 나의 질문에 정확하게 답을 해줘. 문서 내용이 불충분한 경우 '문서에 해당 정보가 없어요'라고 답변해. 문맥 : {context} 질문 : {question}"
)

# 5. RAG 체인 정의 (⭐수정된 부분: retriever에 question만 전달⭐)
rag_chain = ({
    # Lambda 함수를 사용하여 입력 딕셔너리 x에서 'question' 값만 추출하여 retriever에게 전달
    "context": (lambda x: x["question"]) | retriver | format_docs,
    "question": RunnablePassthrough()
}
| rag_prompt
| llm
| StrOutputParser())

# 6. 최종 라우터 체인 구성
final_router_chain = RunnablePassthrough.assign(
    route_info = routing_chain # 질문이 들어오면 먼저 라우팅 판단을 하여 route_info 변수에 저장
) | RunnableBranch(
    # 조건식: route_info의 destination이 "RAG_CHAIN" 인가?
    (lambda x: x['route_info'].destination == "RAG_CHAIN", rag_chain),
    # 조건식: route_info의 destination이 "NORMAL_LLM" 인가?
    (lambda x: x['route_info'].destination == "NORMAL_LLM", normal_llm_chain),
    normal_llm_chain # Fallback: 위에 해당하지 않으면 일반 LLM 실행
)

# 7. 실행 코드 (Invocation)
# RAG 체인으로 가야 하는 질문 (내부 문서 관련)
rag_query = "박 검사장은 언제 서울중앙지검장으로 새로 임명되었나요?"

# 일반 LLM 체인으로 가야 하는 질문 (일반 상식)
normal_query = "인공지능(AI)이란 무엇인가요?"


print("--- 1. RAG Query 실행 (기대 경로: RAG_CHAIN) ---")
rag_result = final_router_chain.invoke({'question': rag_query})
print(f"경로 판단 결과: RAG_CHAIN으로 예상")
print(f"질문: {rag_query}")
print(f"답변: {rag_result}")
print("--------------------------------------------------")

print("--- 2. Normal LLM Query 실행 (기대 경로: NORMAL_LLM) ---")
normal_result = final_router_chain.invoke({'question': normal_query})
print(f"경로 판단 결과: NORMAL_LLM으로 예상")
print(f"질문: {normal_query}")
print(f"답변: {normal_result}")
print("--------------------------------------------------")


ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7d3a4ce42690>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 126, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 99, in _retrieve_info
    info = _metadata.get_service_account_info(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 338, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 263, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http:/

--- 1. RAG Query 실행 (기대 경로: RAG_CHAIN) ---


ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7d3a4ce42690>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 126, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 99, in _retrieve_info
    info = _metadata.get_service_account_info(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 338, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 263, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http:/

경로 판단 결과: RAG_CHAIN으로 예상
질문: 박 검사장은 언제 서울중앙지검장으로 새로 임명되었나요?
답변: 박 검사장은 지난 19일 서울중앙지검장으로 새로 임명됐습니다.
--------------------------------------------------
--- 2. Normal LLM Query 실행 (기대 경로: NORMAL_LLM) ---


ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7d3b123141a0>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 126, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 99, in _retrieve_info
    info = _metadata.get_service_account_info(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 338, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 263, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http:/

경로 판단 결과: NORMAL_LLM으로 예상
질문: 인공지능(AI)이란 무엇인가요?
답변: 안녕하세요! 인공지능(AI)에 대해 궁금하시군요. 친절하게 설명해 드릴게요.

**인공지능(AI)이란 컴퓨터가 사람처럼 생각하고, 학습하고, 문제를 해결하는 능력을 갖도록 만드는 기술을 말해요.**

쉽게 말해, 기계가 인간의 지능적인 행동을 모방하거나 뛰어넘을 수 있도록 설계된 시스템입니다. 여기에는 다음과 같은 능력들이 포함돼요.

*   **학습 능력:** 데이터를 분석해서 패턴을 찾고, 새로운 정보를 학습하며, 경험을 통해 성능을 개선합니다. (예: 많은 사진을 보고 고양이와 강아지를 구분하는 법을 배우는 것)
*   **추론 및 문제 해결:** 주어진 정보를 바탕으로 논리적으로 생각하고, 최적의 해결책을 찾아냅니다. (예: 바둑이나 체스에서 다음 수를 예측하고 승리 전략을 세우는 것)
*   **지각 능력:** 시각(이미지, 영상), 청각(음성), 촉각 등 외부 환경을 인식하고 이해합니다. (예: 얼굴을 인식하거나 음성 명령을 이해하는 것)
*   **자연어 처리:** 사람의 언어를 이해하고 생성하며 대화합니다. (예: 번역 앱, 챗봇)

우리 일상생활 속에서 인공지능은 이미 다양한 형태로 함께하고 있어요. 예를 들어, 스마트폰의 음성 비서 (시리, 빅스비 등), 넷플릭스나 유튜브의 추천 시스템, 자율주행 자동차, 번역 앱, 의료 진단 보조 시스템 등이 대표적인 인공지능 기술의 활용 사례라고 할 수 있죠.

궁극적으로는 인간의 삶을 더 편리하고 풍요롭게 만들고, 복잡한 문제를 효율적으로 해결하는 데 도움을 주는 것이 인공지능의 목표라고 할 수 있습니다.

궁금증이 조금이나마 해소되셨기를 바랍니다! 또 다른 궁금한 점이 있다면 언제든지 물어봐 주세요. 😊
--------------------------------------------------
